In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import ChromeOptions

from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm_notebook as tq
import pandas as pd

In [ ]:
# 에듀윌IT 메인페이지로 접속한다

options = ChromeOptions()
options.add_argument("--start-maximized")
driver = wb.Chrome(options=options)
time.sleep(1)
url = "https://it.eduwill.net/IT2/Main.asp"
driver.get(url)
time.sleep(1)



# 팝업창 닫는다

driver.find_element_by_css_selector("#layerjunsaEvent > div > div.close-set > span:nth-child(2) > a").click()
time.sleep(1)



# 합격수기로 들어간다

driver.find_element_by_css_selector("#eduwillGnb > ul > li:nth-child(2) > a").click()
body = driver.find_element_by_css_selector("body")
for i in range(3):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)


# 합격수기 중 IT자격증 클릭

driver.find_element_by_css_selector("#promotionBoard > div > ul > li:nth-child(10) > a").click()
time.sleep(1)


# 합격수기 가져오기

# 컬럼과 내용 담아줄 리스트 생성하기
ths = []
total_tds = []


# 첫 게시글 선택
driver.find_element_by_css_selector("#listScroll > tbody > tr:nth-child(2) > td.subject > a").click()
time.sleep(1)


# 내용이 있는 tbody로 접근
tbody = driver.find_element_by_css_selector("#detailScroll")


# tbody에서 column이 될 th만 가져오기
for th in tbody.find_elements_by_tag_name("th"):
    ths.append(th.text)
    
    
# 다 가져왔으면 다시 뒤로 가기    
driver.back()


# 다시 첫 게시글로 들어가서 td 가져오고 1페이지 마지막 게시글까지 반복하기
# 반복하기 위해 table에 있는 tbody에 접근하기 ( 게시글 안에 있는 table 아님 )
# 첫 페이지에 있는 게시글 목록의 table에서 tbody임
tbody = driver.find_element_by_xpath("//*[@id=\"listScroll\"]/tbody")
tr = tbody.find_elements_by_tag_name("tr")

# 1페이지에서 tr의 개수만큼 반복문 ( 1페이지 - 29개, 8페이지 - 8개 )
for nth in tq(range(2,len(tr)+1)):
    # td 초기화 저장하고 초기화 시킬 용도
    tds = []
    # 첫 번째 게시글부터 마지막 게시글까지 접속하는 코드
    driver.find_element_by_css_selector("#listScroll > tbody > tr:nth-child("+str(nth)+") > td.subject > a").click()
    # 게시글 접속 후 쉬기
    time.sleep(2)
    # td가 들어있는 tbody 접근
    tbody = driver.find_element_by_css_selector("#detailScroll")
    # tbody에서 td만 가져오기
    for td in tbody.find_elements_by_tag_name("td"):
        tds.append(td.text)
    # tds 리스트에 td값을 다 담아줬다면 total_tds라는 리스트에 리스트 tds를 담아준다
    total_tds.append(tds)
    # 뒤로가기
    driver.back()
    # 좀 쉬어~
    time.sleep(2)
    #다시 위로 가서 두 번째 게시글 들어가고 마지막까지. 이게 1페이지
print("첫 번째 페이지 완료")
    
# n번째 페이지에 들어가서 tr의 개수만큼 반복하자 ( 2페이지 - 29개, 3페이지 - 29개, 8페이지 - 8개 )
for nth in tq(range(3,10)):
    
    # 첫 번째 페이지 다 끝났으니 n번째 페이지로 들어가자 ( 2 ~ 8 페이지 코드로는 3 ~ 9 )
    driver.find_element_by_css_selector("#promotionBoard > div > div.list-table.result > div > a:nth-child("+str(nth)+")").click()
    time.sleep(1)

    # n 번째 페이지에 있는 게시글 목록의 table에서 tbody임
    tbody = driver.find_element_by_css_selector("#listScroll > tbody")
    tr = tbody.find_elements_by_tag_name("tr")

    try:
        for nth in tq(range(2, len(tr)+1)):
            # td 초기화 저장하고 초기화 시킬 용도
            tds = []
            time.sleep(2)
            # 첫 번째 게시글부터 마지막 게시글까지 접속하는 코드
            driver.find_element_by_css_selector("#listScroll > tbody > tr:nth-child("+str(nth)+") > td.subject > a").click()

            # 게시글 접속 후 쉬기
            time.sleep(2)

            # td가 들어있는 tbody 접근
            tbody = driver.find_element_by_css_selector("#detailScroll")

            # tbody에서 td만 가져오기
            for td in tbody.find_elements_by_tag_name("td"):
                tds.append(td.text)

            # tds 리스트에 td값을 다 담아줬다면 total_tds라는 리스트에 리스트 tds를 담아준다
            total_tds.append(tds)

            # 뒤로가기
            driver.back()

            # 좀 쉬어~
            time.sleep(2)
    except:
        print("나 " + str(nth)+ "번째 페이지 첫 번째 게시글이야 빡치지?")
    print(str(nth)+"번째 페이지 완료")
    
print("전체 페이지 완료")
df = pd.DataFrame(total_tds, columns=ths)
df.to_csv("에듀윌_수강후기.csv", encoding='utf-8-sig')